<a href="https://www.kaggle.com/code/mihikadravid/text-sum?scriptVersionId=122123654" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from tqdm import tqdm

pd.options.display.max_colwidth = 200

import warnings
warnings.filterwarnings('ignore')

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames[:5]:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/glove6b/glove.6B.200d.txt
/kaggle/input/glove6b/glove.6B.50d.txt
/kaggle/input/glove6b/glove.6B.300d.txt
/kaggle/input/glove6b/glove.6B.100d.txt
/kaggle/input/nlp-specialization-data/SL_Reviews_Data.csv
/kaggle/input/nlp-specialization-data/Wikipedia_Toxicity_Dataset.csv
/kaggle/input/nlp-specialization-data/singleEnglishWord.wav
/kaggle/input/nlp-specialization-data/Labels_Medical_Notes.csv
/kaggle/input/nlp-specialization-data/harvard.wav
/kaggle/input/nlp-specialization-data/Medical_Notes/Medical_Notes/1893.txt
/kaggle/input/nlp-specialization-data/Medical_Notes/Medical_Notes/1711.txt
/kaggle/input/nlp-specialization-data/Medical_Notes/Medical_Notes/1773.txt
/kaggle/input/nlp-specialization-data/Medical_Notes/Medical_Notes/1812.txt
/kaggle/input/nlp-specialization-data/Medical_Notes/Medical_Notes/1093.txt
/kaggle/input/nlp-specialization-data/pubmed2018_w2v_200D/pubmed2018_w2v_200D/README.txt
/kaggle/input/nlp-specialization-data/pubmed2018_w2v_200D/pubmed2018_w2v_200D

In [2]:
import re

import nltk
from nltk.tokenize import sent_tokenize
from nltk.corpus import stopwords

nltk.download('punkt') # one time execution
nltk.download('stopwords')# one time execution

[nltk_data] Downloading package punkt to /usr/share/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /usr/share/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [3]:
# Read the CSV file
df = pd.read_csv('/kaggle/input/nlp-specialization-data/tennis_articles_v4.csv')
print(df.shape)
df.head(3)

(8, 3)


,article_id,article_text,source
0,1,Maria Sharapova has basically no friends as tennis players on the WTA Tour. The Russian player has no problems in openly speaking about it and in a recent interview she said: 'I don't really hide ...,https://www.tennisworldusa.org/tennis/news/Maria_Sharapova/62220/i-do-not-have-friends-in-tennis-says-maria-sharapova/
1,2,"BASEL, Switzerland (AP), Roger Federer advanced to the 14th Swiss Indoors final of his career by beating seventh-seeded Daniil Medvedev 6-1, 6-4 on Saturday. Seeking a ninth title at his hometown ...",http://www.tennis.com/pro-game/2018/10/copil-stuns-5th-ranked-zverev-to-reach-swiss-indoors-final/77721/
2,3,Roger Federer has revealed that organisers of the re-launched and condensed Davis Cup gave him three days to decide if he would commit to the controversial competition. Speaking at the Swiss Indoo...,https://scroll.in/field/899938/tennis-roger-federer-ignored-deadline-set-by-new-davis-cup


In [4]:
# split the the text in the articles into sentences
sentences = []
for s in df['article_text']:
    sentences.append(sent_tokenize(s))  

In [5]:
# flatten the list
sentences = [y for x in sentences for y in x]

In [6]:
# remove punctuations, numbers and special characters
clean_sentences = pd.Series(sentences).str.replace("[^a-zA-Z]", " ")

# make alphabets lowercase
clean_sentences = [s.lower() for s in clean_sentences]

In [7]:
stop_words = stopwords.words('english')

In [8]:
# function to remove stopwords
def remove_stopwords(sen):
    sen_new = " ".join([i for i in sen if i not in stop_words])
    return sen_new

In [9]:
# remove stopwords from the sentences
clean_sentences = [remove_stopwords(r.split()) for r in clean_sentences]

In [10]:
print(len(clean_sentences))
clean_sentences[:15]

119


['maria sharapova basically friends tennis players wta tour',
 'russian player problems openly speaking recent interview said really hide feelings much',
 'think everyone knows job',
 'courts court playing competitor want beat every single person whether locker room across net one strike conversation weather know next minutes go try win tennis match',
 'pretty competitive girl',
 'say hellos sending players flowers well',
 'uhm really friendly close many players',
 'lot friends away courts',
 'said really close lot players something strategic',
 'different men tour women tour',
 '',
 'think sport mean friends everyone categorized tennis player going get along tennis players',
 'think every person different interests',
 'friends completely different jobs interests met different parts life',
 'think everyone thinks tennis players greatest friends']

In [11]:
from gensim.scripts.glove2word2vec import glove2word2vec
glove_input_file = '/kaggle/input/glove6b/glove.6B.100d.txt'

In [12]:
# Extract word vectors
word_embeddings = {}
f = open(glove_input_file, encoding='utf-8')
for line in f:
    values = line.split()
    word = values[0]
    coefs = np.asarray(values[1:], dtype='float32')
    word_embeddings[word] = coefs
f.close()

In [13]:
len(word_embeddings)

400000

In [14]:
word_embeddings['the'].shape

(100,)

In [15]:
sentence_vectors = []
for i in clean_sentences:
    if len(i) != 0:
        v = sum([word_embeddings.get(w, np.zeros((100,))) for w in i.split()])/(len(i.split())+0.001)
    else:
        v = np.zeros((100,))
    sentence_vectors.append(v)

In [16]:
len(sentence_vectors)

119

The next step is to find similarities among the sentences. We will use cosine similarity to find similarity between a pair of sentences. Let's create an empty similarity matrix for this task and populate it with cosine similarities of the sentences.

In [17]:
# similarity matrix
sim_mat = np.zeros([len(sentences), len(sentences)])
sim_mat.shape

(119, 119)

In [18]:
from sklearn.metrics.pairwise import cosine_similarity

In [19]:
for i in range(len(sentences)):
    for j in range(len(sentences)):
        if i != j:
            sim_mat[i][j] = cosine_similarity(sentence_vectors[i].reshape(1,100), sentence_vectors[j].reshape(1,100))[0,0]

print(sim_mat.shape)
sim_mat[:5,:5]  

(119, 119)


array([[0.        , 0.64378339, 0.59156996, 0.7475453 , 0.63747269],
       [0.64378339, 0.        , 0.83267683, 0.85698557, 0.71764094],
       [0.59156996, 0.83267683, 0.        , 0.82313865, 0.75527066],
       [0.7475453 , 0.85698557, 0.82313865, 0.        , 0.76459354],
       [0.63747269, 0.71764094, 0.75527066, 0.76459354, 0.        ]])

In [20]:
import networkx as nx

nx_graph = nx.from_numpy_array(sim_mat)
scores = nx.pagerank(nx_graph)

In [21]:
ranked_sentences = sorted(((scores[i],s) for i,s in enumerate(sentences)), reverse=True)

In [22]:
# Specify number of sentences to form the summary
sn = 5

# Generate summary
for i in range(sn):
    print(ranked_sentences[i][1])

When I'm on the courts or when I'm on the court playing, I'm a competitor and I want to beat every single person whether they're in the locker room or across the net.So I'm not the one to strike up a conversation about the weather and know that in the next few minutes I have to go and try to win a tennis match.
Major players feel that a big event in late November combined with one in January before the Australian Open will mean too much tennis and too little rest.
Speaking at the Swiss Indoors tournament where he will play in Sundays final against Romanian qualifier Marius Copil, the world number three said that given the impossibly short time frame to make a decision, he opted out of any commitment.
"I felt like the best weeks that I had to get to know players when I was playing were the Fed Cup weeks or the Olympic weeks, not necessarily during the tournaments.
Currently in ninth place, Nishikori with a win could move to within 125 points of the cut for the eight-man event in London 